# Template für Random-Forest Klassifizierung und Regression | Multi-Node/Multi-GPU

***

<img src="./pictures/mog2.PNG"  width="1000px;" hight="900px;">

In diesem Abschnitt wird erkläre, wie man den KMeans Algorithmus auf Worker skaliert und zum Laufen bringt. Dazu nutzen wir Dask und Rapids. Dask bietet uns ein Dashboard wo wir Einzelheiten verfolgen können. Dask-Cuda lässt sich gut mit PyTorch und anderen Frameworks kombinieren, ohne die Clustereinstellungen groß zu ändern.

In dem Nootebook für das Setup wird erklärt, wie man selber sowas auf seinem eigenen PC/Workstation/Cluster installiert. Es gibt auch andere Möglichkeiten das zu installieren. Hier nutzen wir ein Cluster Setup mit Nodes, installiert mittels Docker.   

<u>Hinweis:</u><br>
Wenn es mal hängt, muss ggf. der Cluster, Worker oder das Notebook neu gestartet werden. Sowas passiert, wenn z.B. eine Exception vorkommt.   

<br>

Dask: https://www.dask.org | https://docs.dask.org/en/stable/ <br>
Rapids: https://rapids.ai <br> 

# 1. Rapids und Dask

<img src="./pictures/raspids_oss.PNG"  width="825px;" hight="725px;">
(Bild: (https://on-demand.gputechconf.com/gtcdc/2018/pdf/dc8256-rapids-the-platform-inside-and-out.pdf))

Rapids bietet uns eine Sammlung von Libraries, die uns bei vielen Dingen unterstützen kann. Eine dieser Libraries, die wir nutzen werden, nennt sich cuDF. cuDF sind Cuda Dataframes, die erstmal nur auf einer GPU laufen. Die Idee dahinter ist, dass wir <u>nah zu dasselbe machen</u> wie bei Pandas (Panda DataFrame), aber nur auf einer GPU. Dadurch können viele Berechnungen viel schneller durchgeführt werden. Beispielsweise das Vorverarbeiten von Daten vor dem Trainieren, oder anderweitige Zwecke. Die API ist nah zu identisch wie bei Pandas. 

Rapids funktioniert <u>nur</u> mit <u>Nvidia Grafikkarten</u>.

Rapids ist OpenSource, die API Beschreibungen der Libraries verlinken den dahinterliegenden Code. Der Code enthählt selber nochmal Kommentare und erklärungen.

Rapids entwickelt sich und es kommen Updates sowie Erweiterungen. Algorithmen, die noch nicht (Single-/Multi-Node/ GPU) verfügbar sind, können noch in den neueren Versionen auftauchen. Das gilt besonders für Multi-Node/Multi-GPU Implementierungen der Algorithmen. Es kann sein das manche Multi-Node/Multi-GPU Algorithmen sich am Ende nicht zusammenfassen lassen können, oder das Speichern/Laden sowie die Verwendung dieser auf GPU/CPU eingeschränkt sind. Andere Multi-Node/Multi-GPU Algorithmen können so wie in diesem Template hier verwendet werden. Einge andere können sich in der Anwendung unterscheiden. 


Floating-Point Berechnungen mit cuDF werden auf der GPU parallel ausgeführt, also ist die Reihenfolge nicht deterministisch. Die Ergebnisse können von Pandas abweichen.
a + b ist nicht gleich b + a. Das kann z.B. bei data.sum() vorkommen.
cuDF unterstützt auch verschiedene Datentypen wie int32, int64, float32, float64. Die ganze Liste ist hier zu finden.

Es gibt noch Dask-cuDF, diese nutzt man unter anderem, um große Daten zu Partitionieren. Die API ist Dask-ähnlich. 

Eine weitere Library von Rapids ist cuML, welche auch cuDF nutzt. Diese enthält verschiedene supervised und unsupervsed Algorithmen. Der Fokus hier liegt dabei, dass die Algorithmen möglichst auf einer GPU laufen. Ein Teil dieser Algorithmen kann mittels eines Clusters (cuML nutzt dafür ein Dask Cluster) auf verschiedene Nodes und GPUs skalieren. 

Die API von cuML ist fast genauso aufgebaut wie bei 
Sklearn. <br> Die Release Notes von cuML sind hier zu finden (https://github.com/rapidsai/cuml/releases).
  

Was wir neben Rapids und Dask noch nutzen können ist CuPy [5]. CuPy ist wie Numpy, nur läuft es unter auf GPU. CuPy wird hier nicht detailliert erklärt.

<br> 

cuDF Cuda DataFrames Dok.:  https://docs.rapids.ai/api/cudf/stable/ <br>
cuDF API: https://docs.rapids.ai/api/cudf/stable/api_docs/api/cudf.dataframe/ <br>
cuDF Floating-Point Berechnung: https://docs.rapids.ai/api/cudf/stable/user_guide/pandas-comparison/#floating-point-computation        <br>
cuDF Datentypen: https://docs.rapids.ai/api/cudf/stable/user_guide/data-types/ | https://docs.rapids.ai/api/cudf/stable/user_guide/io/ <br>

cuML GitHub: https://github.com/rapidsai/cuml  <br>
cuML Dok.: https://docs.rapids.ai/api/cuml/stable/ <br>

Dask-cuDF: https://docs.rapids.ai/api/dask-cudf/stable/

CuPy: https://cupy.dev



Was wir nutzen:<br>
cuDF: DataFrames deren API nahezu identisch mit Pandas ist. Also quasi Pandas auf GPU.<br>
dask_cudf: Dask bietet uns DataFrames, die verteilt sind. Mit dask_cudf läuft das ganze auf GPU. Also quasi Dask auf GPU. <br>
cupy: wie Numpy und Scipy aber auf GPU.

<img src="./pictures/df.PNG"  width="725px;" hight="725px;">

Wir teilen das Training in 4 Bereiche auf. Hier nutzen wir MNSG.

<img src="./pictures/rapids_flynn.PNG"  width="725px;" hight="725px;">

Das Bild zeigt in welchen Bereichen man welche DataFrames nutzen kann. Pandas liegt dabei nicht im Fokus.

<img src="./pictures/zuordnung.PNG"  width="1050px;" hight="1050px;">

Da wir mit Dask-cuDF und Dask große Daten lesen und partitionieren können, ist auch der Einsatz im SNSG Bereich möglich. cuDF unterstützt das lesen mehrere Dateien (bis jetzt)  nicht, so das man Dask oder Dask-cuDF nutzen muss, wenn man globstings nutzt, um alle CVS Dateien in einem Verzeichnis zu lesen.

Es gibt auch die Möglichkeit cuDF als Backend für Dask zu nutzen.

<br>

cuDF Backend für Dask: https://docs.rapids.ai/api/dask-cudf/stable/#dataframe-creation-from-on-disk-formats

<b>[Bild Konvertierungen]</b>

<u>Hinweis:</u><br>
Variablen die GPU Speicher nutzen und nicht mehr gebraucht werden, sollten mittels `del` gelöscht werden. Wenn nach dem Löschen immer noch Speicher belegt ist, muss der Kernel des Notebooks neugestartet werden. 

In [1]:
# Imports 
import cupy as   cp
import cudf
import cuml
from cuml.dask.common import utils as dask_utils
import dask_cudf
import dask

import numpy as np
from numpy import genfromtxt

import pandas as pd
import os
import time

Da dier API wie Pandas aufgebaut ist, wird cuDF im detail nicht erklärt. Bei interesse kann man sich die API durchlesen (https://docs.rapids.ai/api/cudf/stable/user_guide/pandas-comparison/).  

In [2]:
# cuDF und Panda series 
s_cuda = cudf.Series([1, 2, cudf.NA])
s_panda = pd.Series([1, 2, pd.NA])

print(f"Pandas:\n{s_cuda}\n")
print(f"Rapids:\n{s_panda}")

Pandas:
0       1
1       2
2    <NA>
dtype: int64

Rapids:
0       1
1       2
2    <NA>
dtype: object


Wir können DataFrames auf verschiedenste Arten erstellen. Unten befinden sich Beispiele wie man DataFrames erstellt. Mehr dazu finden man in der API Beschreibung [3, 6].

Es gibt noch viele weitere Gemeinsamkeiten. Für unsere Zwecke reicht das erstmal.

## 1.1 Start Dask-Cluster (Mit Docker)

Pro GPU wird genau ein Prozess für einen Worker gestartet. Die Worker-ID wird hochgezählt. <br>

<u>Hinweise:</u><br>
Die MNMG Versionen der Algorithmen können auch auf SNSG laufen.

<img src="./pictures/rapids_cluster2.PNG">

In [2]:
# Ein Modul, das uns unterstützt.
import rapids_tools as rapids_tools    

Rapids-tools 1.0
Umgebungsvariablen werden gesetzt
NCCL_SOCKET_NTHREADS: 4
NCCL_NSOCKS_PERTHREAD: 2


In [3]:
## Erstelle Client 
# rapids_tools wird mit dem Cluster geteilt, um Umgebungsvariablen zu setzen.
client = rapids_tools.create_dask_client()  # Gebe IP an. Standard:  127.0.0.1:8786 (Scheduler ist da wo auch Jupyter läuft)
client

Client IP: 127.0.0.1:8786


<Client: 'tcp://149.201.182.203:8786' processes=1 threads=1, memory=62.53 GiB>

Wenn wir eigene Python Module erstellen, können wir diese auch mit dem Cluster teilen.

Beim Ausführen sollte der Status "OK" ausgegeben werden. Wenn die Worker oder der Cluster neugestartet werden, müssen die Module ggf. neu geteilt werden.

In [4]:
# Lade Datei hoch. Gibt Status aus... 
client.upload_file('rapids_tools.py')  # Nur als Beispiel

{'tcp://149.201.182.203:41373': {'status': 'OK'}}

In [6]:
# Zeige Schlüssel. Die Ports ändern sich. 
client.has_what().keys()

dict_keys(['tcp://149.201.182.188:35905', 'tcp://149.201.182.203:38543', 'tcp://149.201.182.205:39657'])

In [20]:
# Starte alle Worker neu
client.restart_workers( client.has_what().keys() )
client.has_what().keys()

dict_keys(['tcp://149.201.182.188:46201', 'tcp://149.201.182.203:33309'])

In [8]:
# Startet client neu.
#client.restart()

# 2. Daten

## 2.1 Daten generieren

Als Anfang erstellen wir Daten. Daten die auf dem Host sind, müssen erst verteilt werden.

cuML bietet uns Möglichkeiten Datensets zu erstellen.

In [5]:
from cuml.datasets  import make_blobs      # Klassifikation    
from cuml.datasets  import make_regression # Regression

from cuml import datasets  
from cuml.model_selection import train_test_split

Ein Weg an Daten zu kommen, ist Datensets zu nutzen, die schon mit den Frameworks kommen. 

Das können wir nutzen, um den Aufbau des Codes zu testen und zu überprüfen, ob die Installation von Rapids und die Dask-Worker funktionieren.

Es gibt einige Parameter, die wir eingeben können, um Daten so zu generieren wie wir es wollen. Um es einfach zu halten, nutzen wir nur wenige Parameter.

<br>

Generiere Single-GPU Daten: https://docs.rapids.ai/api/cuml/stable/api/#dataset-generation-single-gpu

In [6]:
# Trainings- und Testgröße

train_size = 15000000   # 15.000.000  Features: 20 
test_size  = 100
centers    = 5

n_samples  = train_size + test_size

# Für Klassifizierung
X_c, y_c = datasets.make_blobs(n_samples=15000000, n_features=10)
# Für Regression 
#X_r, y_r = datasets.make_regression(n_samples=n_samples)

# Teile Daten auf
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split( X_c, y_c, train_size=0.80, random_state=5)
#X_train_r, X_test_r, y_train_r, y_test_r = train_test_split( X_r, y_r, train_size=0.80, random_state=5)

In [8]:
X_c[1]

array([ 0.76812583, -3.785273  ,  1.8216226 , 10.967538  , -1.482454  ,
       -4.4057393 , -1.8101425 , -9.3877735 , -2.7475626 , -6.707412  ],
      dtype=float32)

Jetzt können wir uns anschauen was gemacht wurde, so könnten unsere Trainingsdaten auch aussehen.

In [10]:
print(f"Daten Klassifizierung:\nX: {type(X_c)} y: {type(y_c)} \n X[0]: {X_c[0]}  \n y[0]: {y_c[0]} \n Länge X, y: {len(X_c)}, {len(y_c)}")

Daten Klassifizierung:
X: <class 'cupy.ndarray'> y: <class 'cupy.ndarray'> 
 X[0]: [-4.390422 -7.993816]  
 y[0]: 0.0 
 Länge X, y: 2100, 2100


In [11]:
print(f"Daten Regression:\nX: {type(X_r)} y: {type(y_r)} \n X[0]: {X_r[0]}  \n y[0]: {y_r[0]} \n Länge X, y: {len(X_r)}, {len(y_r)}")

Daten Regression:
X: <class 'cupy.ndarray'> y: <class 'cupy.ndarray'> 
 X[0]: [0.27291918 0.6470641 ]  
 y[0]: [34.67788] 
 Länge X, y: 2100, 2100


Die erstellten Daten sind in CuPy Format. Für das Trainieren müssen die Daten im Dask-cuDF Format sein. Alternativ können die Daten im Format Dask Array sein, mit CuPy als Backend (Siehe fit() https://docs.rapids.ai/api/cuml/stable/api/#id43).

## 2.2 Dask und Dask_cuDF

Schauen wir uns den Fall an, das die Daten <u>erst verteilt werden</u> müssen. 

In [12]:
# X_c ist auf dem Host und nicht Partitioniert.
X_c

array([[-4.390422 , -7.993816 ],
       [-1.7899303, -8.560305 ],
       [ 6.878088 ,  3.2281156],
       ...,
       [ 7.9013543,  3.418171 ],
       [-3.146109 , -8.004911 ],
       [-4.038967 , -7.6395087]], dtype=float32)

Bei Verteilen oder Laden der Daten mit HDFS, können wir eine Chunkgröße angeben oder die Anzahl der Partitionen. Die Worker übernehmen eine Anzahl der Partitionen für das Trainieren.

X ist ein cupy. Wir brauchen ein dask_cuDF oder ein Dask Array mit cupy als Backend. 

Erst wandeln wir das Array X in ein DataFrame um. Dann werden wir X partitionieren.

In [21]:
## cupy -> cuDF
X_cudf_c = cudf.DataFrame(X_train_c).astype(np.float32)
y_cudf_c = cudf.Series(y_train_c).astype('int32')
print(f"X Type: {type(X_cudf_c)} \nX ist:\n {X_cudf_c}\ny Type: {type(y_cudf_c)}\ny ist:\n{y_cudf_c}")

X Type: <class 'cudf.core.dataframe.DataFrame'> 
X ist:
                  0         1         2         3          4         5  \
0         0.445445 -4.056909  2.238461  9.445652  -3.096715 -4.307353   
1        -8.047665 -2.113462 -3.302359  9.171656  -1.997621  9.484214   
2         0.394223 -4.543429  1.361037  9.055886  -2.092587 -2.869893   
3        -7.687822  0.303481  3.407153 -3.149281   8.906869 -1.914889   
4        -7.777630 -3.561938 -5.610600  9.720518  -0.735540  6.552356   
...            ...       ...       ...       ...        ...       ...   
11999995 -6.935137  1.793511  3.898031 -6.972420   9.803249 -2.236868   
11999996 -8.591798  1.281032  3.008148 -4.796408  10.242216 -2.150061   
11999997 -8.250855 -1.799571 -3.471779  8.979384  -1.971615  9.086596   
11999998 -5.774130  1.029114  3.132514 -6.744684   6.761896 -0.117011   
11999999 -1.915761 -3.922513  2.279961  9.516230  -1.955437 -3.873719   

                 6          7         8         9  
0        -0.72

Danach werden wir das DataFrame Partitionieren.

In [14]:
# Dasselbe bei für _r, ohne astype(np.float32)
X_cudf_r = cudf.DataFrame(X_train_r)
y_cudf_r = cudf.Series(y_train_r)

In [22]:
workers = client.has_what().keys()
n_workers = len(workers)
print(f"nworker: {n_workers}")

nworker: 2


In [23]:
# cuDF -> dask_cuDF
X_dask_c = dask_cudf.from_cudf( X_cudf_c, npartitions=n_workers )  # oder  npartitions = n_workers
y_dask_c = dask_cudf.from_cudf( y_cudf_c, npartitions=n_workers )

X_dask_c, y_dask_c = dask_utils.persist_across_workers(client, [X_dask_c, y_dask_c], workers=client.has_what().keys())  # Halte Ergebnis im Speicher 

In [19]:
del X_dask_c, y_dask_c, X_cudf_c, y_cudf_c

In [17]:
# Für _r auch
X_dask_r = dask_cudf.from_cudf( X_cudf_r, npartitions=n_workers )  # oder  npartitions = n_workers
y_dask_r = dask_cudf.from_cudf( y_cudf_r, npartitions=n_workers )

X_dask_r, y_dask_r = dask_utils.persist_across_workers(client, [X_dask_r, y_dask_r], workers=client.has_what().keys())  # Halte Ergebnis im Speicher 

In [18]:
print(f"dask_X_c: {type(X_dask_c)}\n{X_dask_c}")

dask_X_c: <class 'dask_cudf.core.DataFrame'>
<dask_cudf.DataFrame | 3 tasks | 3 npartitions>


In [19]:
X_dask_c

,0,1
npartitions=3,,
0,float32,float32
560,...,...
1120,...,...
1679,...,...


In [20]:
## Damit sehen wir was der Cluster hat.
# Jeder hat ein Teil des DataFrames. Jeder Worker nutzt die eigenen Partitionen. 
client.has_what()

{'tcp://149.201.182.188:33351': ("('from_cudf-427dd0072f4a48c130c282242402dc99', 1)",
  "('from_cudf-a33c3ddacdcb428237dc320102bb7c71', 1)",
  "('from_cudf-10f10ff11cf8696158b024e810bfbef0', 1)",
  "('from_cudf-ec2bd4900dbe7e4b6dafc3178d92b077', 1)"),
 'tcp://149.201.182.203:46815': ("('from_cudf-427dd0072f4a48c130c282242402dc99', 0)",
  "('from_cudf-a33c3ddacdcb428237dc320102bb7c71', 0)",
  "('from_cudf-10f10ff11cf8696158b024e810bfbef0', 0)",
  "('from_cudf-ec2bd4900dbe7e4b6dafc3178d92b077', 0)"),
 'tcp://149.201.182.205:37759': ("('from_cudf-427dd0072f4a48c130c282242402dc99', 2)",
  "('from_cudf-a33c3ddacdcb428237dc320102bb7c71', 2)",
  "('from_cudf-10f10ff11cf8696158b024e810bfbef0', 2)",
  "('from_cudf-ec2bd4900dbe7e4b6dafc3178d92b077', 2)")}

So siet der Flow aus, wenn wir Daten als Host erzeugen und dann verteilen.

<img src="./pictures/part.PNG"  width="925px;" hight="925px;">

Zusammeengefasst: <br>
Wenn wir Daten als Host erstellen (z.B. um den Aufbau zu testen), müssen die Daten Partitioniert werden. Das verteilen geht gut mit <u>geringen Datenmengen</u>.

Nach dem Verteilen hat jeder Worker ein Teil dieser Daten. Diese Teile sind Partitionen.

<img src="./pictures/part2.PNG"  width="925px;" hight="925px;">

## 2.3 Daten sind schon Lokal verfügbar

Mit Dask-cuDF können wir Daten direkt in die GPU laden und Daten schreiben. Es gibt viele Parameter, die angegeben werden können (https://docs.rapids.ai/api/dask-cudf/stable/api/#creating-and-storing-dataframes). 

Wenn die Daten nicht in den Speicher passen, können diese partitioniert werden, indem wir die Daten mit Dask-cuDF laden.

<img src="./pictures/rapids_chunk.PNG">

Wir können Daten mit Dask laden und daraus ein Dask-cuDF erstellen.

Intern nutzt jeder Worker cuDF um die Daten zu lesen. Es können verschiedene Quellen angegeben werden wie HDFs, S3, http, ... (https://docs.dask.org/en/stable/how-to/connect-to-remote-data.html)

Wir können auch direkt Dask-cuDF nutzen um Daten zu laden. Das geht genau so wie bei Dask. Das schreiben der Daten mit Dask-cuDF folg auch der gleichen API.

Jeder Worker sollte die gleichen Pfade sehen können.

Das erstellte Dask-cuDF kann dann zum trainieren genutzt werden. 

<br>

Rapids Dask-cuDF Creating and storing DataFrames:  https://docs.rapids.ai/api/dask-cudf/stable/api/ <br>
Reading Larger than Memory CSVs with RAPIDS and Dask: https://medium.com/rapids-ai/reading-larger-than-memory-csvs-with-rapids-and-dask-e6e27dfa6c0f


<u>Hinweis zu HDFS:</u><br>
Das lesen von vielen Dateien oder einer großen Datei bringt keinen Vorteile der Datenlokalität mehr, siehe (https://stackoverflow.com/questions/54565738/does-dask-communicate-with-hdfs-to-optimize-for-data-locality)<br>
Das kann sich noch ändern. 
 

# 3. Multi-Node/Multi-GPU Training: Dask und Dask_cuDF

Wie bereits erwähnt, ist das Interface fast genau so wie bei sklearn. Wenn wir jetzt Daten haben und diese noch verarbeiten wollen, muss darauf geachtet werden, dass die Datentypen am Ende richtig sind. 

Für das Model gibt es einige Parameter, die wir einstellen können, dieser Auszug zeigt einige Parameter:  <br>

Random-Forest Klassifikation:<br>
`split_criterion: =   (0 or 'gini' | 1 or 'entropy' ) ` <br>
`max_depthint= (default = 16) ` <br>
`max_leavesint= (default = -1)` <br>
`n_binsint= (default = 128)` <br>
`n_streams=4 Number of parallel streams used for forest building. ` <br>
Für Regression: <br>
`split_criterion: =   ( 2 or 'mse' | 4 or 'poisson' | 5 or 'gamma' | 6 or 'inverse_gaussian') `<br>
`accuracy_metric= ( r2 | median_ae | mean_ae | mse ) Decides the metric used to evaluate the performance of the model. ` <br>
`max_depth= (default = 16) ` <br>
`max_leaves= (default = -1)` <br>
`n_bins= (default = 128)` <br>
`n_streams=4 Number of parallel streams used for forest building. ` <br>


Wir werden die normalen Einstellungen beibehalten.

<br>

Sklearn Random-Forest: <br>
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.htmll <br>
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html <br>
cuML MNMG Random-Forest: https://docs.rapids.ai/api/cuml/stable/api/#id47

In [24]:
# Imortiere Multi-Node RM
from cuml.dask.ensemble import RandomForestClassifier as multi_RandomForestClassifier
from cuml.dask.ensemble import RandomForestRegressor  as multi_RandomForestRegressor

Jetzt kann man wie üblich vorgehen. Das Model Trainieren, Speichern, Laden und Evaluieren.

Wir können das Dashboard von Dask öffnen und sehen unter GPU die Auslastung (wenn es nicht zu schnell fertig ist).

In [25]:
# Erstelle Model | RM Klassifizierung 
multi_rm_c = multi_RandomForestClassifier() 
                     
# Training  
%time multi_rm_c.fit(X_dask_c, y_dask_c)
# 1gpu: 48s   2gpu: 12s   3gpus

CPU times: user 69.7 ms, sys: 3.8 ms, total: 73.5 ms
Wall time: 11.9 s


In [23]:
# Erstelle Model | RM Regressor 
multi_rm_r = multi_RandomForestRegressor()
                     
# Training  
# - Input Type
%time multi_rm_r.fit(X_dask_r, y_dask_r)

CPU times: user 55 ms, sys: 3.65 ms, total: 58.7 ms
Wall time: 227 ms


Das Model ist momentan noch verteilt, wir können trotzdem Predictions machen. Dafür müssen wir die Daten wieder verteilen, wenn diese nicht schon lokal vorliegen.

In [24]:
# Regression 
to_predict_r = X_test_r
print(f"Länge: {len(to_predict_r)} \nTyp: {type(to_predict_r)}\n {to_predict_r}")

Länge: 420 
Typ: <class 'cupy.ndarray'>
 [[ 1.42820692e+00 -1.50018811e+00]
 [-1.86547264e-01  6.84669912e-01]
 [-1.22138178e+00 -2.04375267e-01]
 [-6.40086412e-01 -3.77113789e-01]
 [-6.18870735e-01 -8.98498952e-01]
 [ 1.13327253e+00 -1.53003788e+00]
 [ 8.95731211e-01  6.25492394e-01]
 [ 1.14272475e+00  2.14870214e+00]
 [-4.42176908e-01  4.91543720e-03]
 [-5.99980056e-02 -3.41275901e-01]
 [-4.65737522e-01  1.05170631e+00]
 [ 1.65781474e+00  4.48984802e-01]
 [-8.83426607e-01  9.25974995e-02]
 [ 1.60494137e+00  3.69798005e-01]
 [-2.74298728e-01 -4.47603688e-03]
 [ 1.14247286e+00 -1.59782290e+00]
 [-5.26987076e-01 -7.28040189e-02]
 [ 2.78331017e+00 -1.07615638e+00]
 [ 2.61367291e-01 -1.33413446e+00]
 [-1.22379982e+00 -9.01313066e-01]
 [-1.17765772e+00  1.93844810e-02]
 [-1.42233443e+00 -9.62353289e-01]
 [ 1.75954118e-01 -8.20863843e-01]
 [-1.21787906e+00 -1.77918777e-01]
 [-8.94374728e-01  1.12684555e-01]
 [-6.45377457e-01  1.32797241e+00]
 [-8.06818426e-01  4.02854800e-01]
 [ 4.32281911e

Das cupy Array müssen wir in ein cuDF, dann in ein dask_cuDF umwandeln.

In [25]:
# Regression 
to_predict__dask_r = dask_cudf.from_cudf(cudf.DataFrame( to_predict_r ), npartitions=n_workers)

In [26]:
# Regression 
pred_futures = multi_rm_r.predict(to_predict__dask_r)
print(pred_futures)

<dask_cudf.Series | 10 tasks | 3 npartitions>


Die futures wurden erstellt. Um die Berechnung zu starten schreiben wir `.compute()`

In [27]:
%time pred_futures__MultiNodeOrig_results_r = pred_futures.compute()

CPU times: user 33.1 ms, sys: 54 µs, total: 33.2 ms
Wall time: 405 ms


In [28]:
# Index of table | result
pred_futures__MultiNodeOrig_results_r.head(10)

0    -66.669022
1     30.539501
2    -21.801332
3    -24.911106
4    -49.476677
5    -68.462410
6     38.065460
7    115.754089
8     -4.368572
9    -17.929943
dtype: float32

Dasselbe ist auch mit dem Klassifizierer möglich.

In [29]:
# Klassifizierung
to_predict_c = X_test_c
print(f"Länge: {len(to_predict_c)} \nTyp: {type(to_predict_c)}\n {to_predict_c} \n")
# Klassifizierung
to_predict__dask_c = dask_cudf.from_cudf(cudf.DataFrame( to_predict_c ), npartitions=n_workers)
pred_futures_c = multi_rm_c.predict(to_predict__dask_c)
# Klassifizierung
%time pred_futures__MultiNodeOrig_results_c = pred_futures_c.compute()
pred_futures__MultiNodeOrig_results_c.head(10)

Länge: 420 
Typ: <class 'cupy.ndarray'>
 [[ -3.8746302   -7.7102246 ]
 [  8.461737     3.488795  ]
 [ -2.9855368  -10.363907  ]
 [ -4.3325787   -8.440752  ]
 [  8.542121     4.9818277 ]
 [ -1.9604208   -8.873977  ]
 [ -2.7259488   -8.845068  ]
 [  7.168546     3.9781618 ]
 [  8.437181     4.601076  ]
 [ -2.8101258   -9.039558  ]
 [  6.8697004    1.895816  ]
 [  5.106079     5.1688995 ]
 [  5.723507     2.9781814 ]
 [  7.0216484    3.4909797 ]
 [ -2.0167768   -9.756744  ]
 [ -2.0137606   -7.268175  ]
 [  7.0262985    3.896807  ]
 [ -3.444375    -9.146772  ]
 [  8.320475     1.4232137 ]
 [ -4.5000396   -8.577863  ]
 [  7.446222     4.237891  ]
 [  6.7105694    1.651472  ]
 [  7.0225286    1.8492386 ]
 [  8.167989     4.9849973 ]
 [ -4.529224    -7.750746  ]
 [  9.288561     3.8546402 ]
 [ -3.1707718   -9.65032   ]
 [ -2.4145374  -10.171262  ]
 [ -4.390422    -7.993816  ]
 [  8.872564     2.2624142 ]
 [  7.053499     2.39192   ]
 [ -4.0264945   -8.507057  ]
 [ -2.5105553   -8.750834  ]
 [

0    0.0
1    1.0
2    0.0
3    0.0
4    1.0
5    0.0
6    0.0
7    1.0
8    1.0
9    0.0
dtype: float32

# 3. Speichern, Laden, Testen

Das Speichern bei MNMG geht etwas anders, aber auch schnell und einfach. Dazu muss das verteilte Model zusammengefasst werden.

Multi-Node / Multi-GPU => Single-Node/Single-GPU Model

<br>

Speichern und Laden: https://docs.rapids.ai/api/cuml/stable/pickling_cuml_models/

In [31]:
combined_c = multi_rm_c.get_combined_model()
combined_r = multi_rm_r.get_combined_model()

In [32]:
# Speichere Single-Node/Single-GPU Model 
import pickle
# RM Klassifizierer
pickle.dump(combined_c, open("template_data_RM_Multi/cuml_SingleNode_RM_c_model.pkl", "wb"))
# RM Regressor
pickle.dump(combined_r, open("template_data_RM_Multi/cuml_SingleNode_RM_r_model.pkl", "wb"))

# Lade:
loaded_model_c = pickle.load(open("template_data_RM_Multi/cuml_SingleNode_RM_c_model.pkl", "rb"))
loaded_model_r = pickle.load(open("template_data_RM_Multi/cuml_SingleNode_RM_r_model.pkl", "rb"))

Das Predicten sollte dann ohne Probleme funktionieren.

In [34]:
# Klassifizierung
loaded_model_c.predict(cp.asarray( [ [1.5, 2.4] ], dtype=cp.float32 ))

array([2.], dtype=float32)

In [35]:
# Regression
loaded_model_r.predict(cp.asarray( [ [1.5, 2.4] ], dtype=cp.float32 ))

array([124.833855], dtype=float32)

Danach können wir Metriken nutzen.

In [41]:
# Auch mit cuML möglich
#   cuml.metrics.accuracy_score()    # GPU
#   cuml.metrics.r2_score()          # GPU

from sklearn.metrics import accuracy_score   # CPU
from sklearn.metrics import r2_score         # CPU

Konvertiere cuDF zu Numpy:

In [39]:
pred_c_numpy = cudf.DataFrame.to_numpy(pred_futures__MultiNodeOrig_results_c)
pred_r_numpy = cudf.DataFrame.to_numpy(pred_futures__MultiNodeOrig_results_r)

In [123]:
# Eval KNN Klassifizierer 
score__c = accuracy_score( y_test_c.get(), loaded_model_c.predict(X_test_c).get())

# Eval KNN Regressor 
score__r = r2_score( y_test_r.get(), loaded_model_r.predict(X_test_r).get() )
print(f"score__c: {score__c} \nscore__r: {score__r}")

score__c: 0.8333333333333334 
score__r: 0.9927694214601607


Wir können wieder Predictions machen.

In [133]:
pred__SingleLoadedModeOrig_results_c = loaded_model_c.predict(to_predict_c)
pred__SingleLoadedModeOrig_results_r = loaded_model_r.predict(to_predict_r)

In [134]:
pred__SingleLoadedModeOrig_results_r

array([-6.66690216e+01,  3.05395012e+01, -2.18013325e+01, -2.49111061e+01,
       -4.94766769e+01, -6.84624100e+01,  3.80654602e+01,  1.15754089e+02,
       -4.36857176e+00, -1.79299431e+01,  4.94022369e+01,  3.54106102e+01,
       -3.95462489e+00,  2.90549774e+01, -2.64627433e+00, -7.13911133e+01,
       -9.37537193e+00, -4.07250633e+01, -6.52710114e+01, -5.25899887e+01,
       -8.81695747e+00, -5.84501266e+01, -3.92684212e+01, -2.09454613e+01,
       -3.81798983e+00,  5.94091377e+01,  1.33209267e+01,  4.03186836e+01,
        3.44993439e+01, -7.65407486e+01,  7.07252655e+01, -3.01502972e+01,
       -7.15061798e+01,  2.80474281e+01, -4.60887947e+01,  8.25705948e+01,
        7.59161911e+01, -5.99445877e+01, -1.87173424e+01, -5.82415276e+01,
        8.70534611e+00, -3.46351929e+01,  9.04294872e+00, -1.22446640e+02,
       -4.83229790e+01,  5.15524025e+01,  1.87793446e+01,  4.50079193e+01,
        4.46109009e+01, -1.09434357e+02, -6.60757599e+01, -1.61232929e+01,
       -5.73884621e+01,  

## 3.1 Multi-Node/-Multi-GPU Predictions

Das trainierte Model ist momentan nur für den Single-GPU Bereich einsetzbar. Um die Predictions zu verteilen kann man.:<br>
1) Jedem Worker das Model geben und die Daten hineinladen.



### 3.1.1 Details

Das gespeicherte Model befindet sich auf dem Host oder wurde schon verteilt. In dem Fall, dass sich das Model auf dem Host befindet, muss es an jeweilige Worker verteilt werden.

In [45]:
# Host lädt Model
loaded_model_c = pickle.load(open("template_data_RM_Multi/cuml_SingleNode_RM_c_model.pkl", "rb"))
loaded_model_c

RandomForestClassifier()

Mit Dask können wir das Model an alle Worker senden.

In [46]:
futures = client.scatter(loaded_model_c, broadcast=True)

In [47]:
# Future sollte den status finished angeben
futures

<Future: finished, type: cuml.ensemble.randomforestclassifier.RandomForestClassifier, key: RandomForestClassifier-5032de5a7a220fae390d0303c00816f5>

Jetzt können wir eine Funktion schreiben die uns die Predictions liefert. Wir können Lokale Daten nehmen oder erst wieder Daten für die Prediction verteilen.

In [51]:
to_predict__dask = dask_cudf.from_cudf(cudf.DataFrame( X_test_c ), npartitions=n_workers)
to_predict__dask

,0,1
npartitions=3,,
0,float32,float32
140,...,...
280,...,...
419,...,...


<img src="./pictures/pred1.PNG"  width="925px;" hight="925px;">

<img src="./pictures/pred2.PNG"  width="325px;" hight="225px;" >

Mit Dask Delayed sagen wir, dass diese Funktion parallel ausgeführt werden soll. In der Funktion können wir verteilt Daten lesen und bearbeiten, bevor wir die Predictions machen. 

<br>

Dask Delayed: https://docs.dask.org/en/stable/delayed.html

In [49]:
# Länge der Testdaten
len(X_test_c)

420

Wir haben die Daten als Dask-cuDF gelesen. Wenn wir das als Host machen, müssen wir in einer Schleife die Partitionen übergeben.

In [119]:
@dask.delayed
def dask_predict(test_data, part):  # Jeder Worker führt das aus. 
    
    # Das wird in der Console ausgegeben. Damit sehen wir das alles passt, und jeder worker seine Partitionen nimmt. 
    print(f"type: {type(test_data)} länge: {len(test_data)}\nHead:\n{test_data.head(2)}\nPartition: {part}")

    predictions = loaded_model_c.predict(test_data) 

    return predictions

Wir müssen die Anzahl der Partitionen angeben. Die Worker werden die eigenen Partitionen bearbeiten.

In [120]:
results = []
parts = 3   # Anzahl Partitionen

for part in range(parts):
    fut = dask_predict( to_predict__dask.get_partition(part), part )
    results.append( fut )
    
delayed_results = dask.compute(*results)

print(f"Type: {type(delayed_results)}\n")

Type: <class 'tuple'>



In [121]:
# Mache daraus ein Numpy Array.
deleayed_pred_c = []
for i in range( len(delayed_results) ):
    deleayed_pred_c = np.append(deleayed_pred_c, np.asarray( delayed_results[i].to_numpy()) )
type(deleayed_pred_c)

numpy.ndarray

Danach werden wir überprüfen, ob die gemachten Predictions gleich sind.

In [122]:
## Überprüfe ob das Array pred_futures__MultiNodeOrig_results == deleayed_pred ist.
# cp.asnumpy() formt das Array in Numpy um.
rapids_tools.arrays_equal( cp.asnumpy( pred__SingleLoadedModeOrig_results_c ),  deleayed_pred_c )

Gebe 10 Zeilen aus, die ungleich sind


Eine andere Möglichkeit ist, dass die Worker die Daten lokal lesen und ggf. auch für sich diese Partitionieren. 

Es können wieder veschiedene Quellen angegeben werden. Für ein einfaches kleines Beispiel, schreibt jeder Worker eine CSV Datei die später gelesen wird. 



In [140]:
## Diese Datei wird im lokalen Verzeichnis geschrieben.
# Regression
np.savetxt('csv_file.csv', cp.asnumpy( X_test_r ), delimiter=",", header="A,B") # Sonst werden die Daten als header genommen 

In [141]:
## Damit schreibt jeder Worker diese Datei.
@dask.delayed    # Jeder Worker führt das aus. 
def dask_write(test_data):
    print("Write file")
    np.savetxt('csv_file.csv', np.asarray(test_data, dtype=np.float32), delimiter=",", header="A,B")
    
futs = []
for i in range(3):
    futs.append(dask_write( cp.asnumpy( X_test_r ) ))
dask.compute(*futs)

(None, None, None)

jetzt können wir diese CSV Datei laden und für das Testen nutzen.
Am Ende bekommen wir wieder ein Tupel. Diesemal werden alle Daten genutzt.

In [147]:
@dask.delayed
def dask_predict_2():

    test_data = cudf.read_csv("csv_file.csv")       # Lese lokale Daten
    test_data = test_data.astype(dtype='float32')   # Muss float32 sein, oder ändere IO
    
    # oder- nutze Dask-cuDF um lokale Daten zu partitionieren. 
    #test_data = dask_cudf.read_csv("csv_file.csv",  blocksize="5KB")       # Lese lokale Daten, gebe Blocksize an
    #test_data = test_data.astype(dtype='float32')                          # Muss float32 sein, oder ändere IO
    
    print(f"Data loaded, len: {len(test_data)}")
    
    predictions = loaded_model_r.predict(test_data)
    return predictions

In [148]:
futs = []
for i in range(3):
    futs.append(dask_predict_2())

delayed_results = dask.compute(*futs)
delayed_results

(0    -66.669022
 1     30.539501
 2    -21.801332
 3    -24.911106
 4    -49.476677
         ...    
 23   -22.178020
 24   -27.109999
 25    61.012112
 26    90.758492
 27    65.508560
 Length: 420, dtype: float32,
 0    -66.669022
 1     30.539501
 2    -21.801332
 3    -24.911106
 4    -49.476677
         ...    
 23   -22.178020
 24   -27.109999
 25    61.012112
 26    90.758492
 27    65.508560
 Length: 420, dtype: float32,
 0    -66.669022
 1     30.539501
 2    -21.801332
 3    -24.911106
 4    -49.476677
         ...    
 23   -22.178020
 24   -27.109999
 25    61.012112
 26    90.758492
 27    65.508560
 Length: 420, dtype: float32)

Man sieht das alle Worker alle Predictions gemacht haben.

In [150]:
# Mache daraus ein Numpy Array.
deleayed_pred_r = []
for i in range( 1 ):
    deleayed_pred_r = np.append(deleayed_pred_r, np.asarray( delayed_results[i].to_numpy()) )
type(deleayed_pred_r)

numpy.ndarray

In [151]:
rapids_tools.arrays_equal( cp.asnumpy( pred__SingleLoadedModeOrig_results_r ),  deleayed_pred_r )

Gebe 10 Zeilen aus, die ungleich sind


## 3.2 Treelite

Jetzt schauen wir uns den Fall an, dass das Zielsystem keine GPU hat. Wir trainieren das Model mit GPUs und nutzen es in einem anderen System, dass keine GPUs besitzt. Für solche fälle können wir Random-Forest in einen allgemeinen Baum umwandeln.

Auf dem Zielsystem muss dann nur die Library Treelite und numpy installiert sein.

Das cuML Model bietet Intern schon Möglichkeiten Treelite Attribute zu extrahieren. 

<br>

Treelite Dok.: https://treelite.readthedocs.io/en/latest/ <br>
Wandel cuMl RM in Treelite um: https://docs.rapids.ai/api/cuml/nightly/pickling_cuml_models/#Exporting-cuML-Random-Forest-models-for-inferencing-on-machines-without-GPUs

In [152]:
# Gebe ein Checkpoint Pfad an
checkpoint_path = './template_data_RM_Multi/treelite_single_c.tl'
# Klassifikation
combined_c.convert_to_treelite_model().to_treelite_checkpoint(checkpoint_path)

# Gebe ein Checkpoint Pfad an
checkpoint_path = './template_data_RM_Multi/treelite_single_r.tl'
# Regression
combined_r.convert_to_treelite_model().to_treelite_checkpoint(checkpoint_path)

Danach muss das Model auf das Zielsystem kopiert werden. Dort sollte die Library treelite installiert sein, z.B. mit `pip install treelite`. Rapids bzw. cuML muss nicht vorhanden sein.

In [153]:
# Auf dem Zielsystem #

# Treelite 
import treelite

# The checkpoint file has been copied over
checkpoint_path = './template_data_RM_Multi/treelite_single_c.tl'
tl_model_c = treelite.Model.deserialize(checkpoint_path)

# Regression
checkpoint_path = './template_data_RM_Multi/treelite_single_r.tl'
tl_model_r = treelite.Model.deserialize(checkpoint_path)

In [154]:
treelite.gtil.predict(tl_model_r, np.asarray([[1., 2.]],  dtype=np.float32))

array([[105.74283]], dtype=float32)

Wir können wieder die Genauigkeit testen.

<u>Hinweis:</u>
In einer älteren Version von Rapids (23.04.00) gibt es ein Bug der dazu führt, dass der Baum des Klassifizierer nicht richtig zusammengeführt wird, was zu einem schlechten Score führt.<br>
Siehe: https://github.com/rapidsai/cuml/issues/5359 und https://github.com/rapidsai/cuml/pull/5387  

In [155]:
predictions_c_treelite = treelite.gtil.predict(tl_model_c, np.asarray(X_test_c.get()))
score__c      = accuracy_score( y_test_c.get(), predictions_c_treelite )

predictions_r_treelite = treelite.gtil.predict(tl_model_r, np.asarray(X_test_r.get()))
score__r      = r2_score( y_test_r.get(), predictions_r_treelite )


print(f"score_c: {score__c} \nscore_r: {score__r}")

score_c: 0.3738095238095238 
score_r: 0.9927693920195462


In [163]:
rapids_tools.arrays_almost_equal( cp.asnumpy( pred__SingleLoadedModeOrig_results_r ),  predictions_r_treelite )

AssertionError: 
Arrays are not almost equal to 4 decimals

Mismatched elements: 3 / 420 (0.714%)
Max absolute difference: 0.00018311
Max relative difference: 2.2971537e-06
 x: array([-6.6669e+01,  3.0540e+01, -2.1801e+01, -2.4911e+01, -4.9477e+01,
       -6.8462e+01,  3.8065e+01,  1.1575e+02, -4.3686e+00, -1.7930e+01,
        4.9402e+01,  3.5411e+01, -3.9546e+00,  2.9055e+01, -2.6463e+00,...
 y: array([-6.6669e+01,  3.0539e+01, -2.1801e+01, -2.4911e+01, -4.9477e+01,
       -6.8462e+01,  3.8065e+01,  1.1575e+02, -4.3686e+00, -1.7930e+01,
        4.9402e+01,  3.5411e+01, -3.9546e+00,  2.9055e+01, -2.6463e+00,...

In [164]:
client.shutdown()